In [1]:
# !pip install opencv-python pygame

In [2]:
import cv2
import pygame
import random
import time
import pandas as pd
from IPython.display import clear_output
import os

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
num_balls = 3
type_visual = "none"
# Possible types are: none, arrows, eyes, text

three_ball_none = ["./3Ball/3Ball_No_Screen1.mp4", "./3Ball/3Ball_No_Screen2.mp4", "./3Ball/3Ball_No_Screen3.mp4"]
three_ball_arrows = ["./3Ball/3Ball_Arrows1.mp4", "./3Ball/3Ball_Arrows2.mp4", "./3Ball/3Ball_Arrows3.mp4"]
three_ball_eyes = ["./3Ball/3Ball_Eyes1.mp4", "./3Ball/3Ball_Eyes2.mp4", "./3Ball/3Ball_Eyes3.mp4"]
three_ball_text = ["./3Ball/3Ball_Text1.mp4", "./3Ball/3Ball_Text2.mp4", "./3Ball/3Ball_Text3.mp4"]

five_ball_none = ["./5Ball/5Ball_No_Screen1.mp4", "./5Ball/5Ball_No_Screen2.mp4", "./5Ball/5Ball_No_Screen3.mp4", 
                  "./5Ball/5Ball_No_Screen4.mp4", "./5Ball/5Ball_No_Screen5.mp4"]
five_ball_arrows = ["./5Ball/5Ball_Arrows1.mp4", "./5Ball/5Ball_Arrows2.mp4", "./5Ball/5Ball_Arrows3.mp4", 
                    "./5Ball/5Ball_Arrows4.mp4", "./5Ball/5Ball_Arrows5.mp4"]
five_ball_eyes = ["./5Ball/5Ball_Eye1.mp4", "./5Ball/5Ball_Eye2.mp4", "./5Ball/5Ball_Eye3.mp4", 
                  "./5Ball/5Ball_Eye4.mp4", "./5Ball/5Ball_Eye5.mp4"]
five_ball_text = ["./5Ball/5Ball_Text1.mp4", "./5Ball/5Ball_Text2.mp4", "./5Ball/5Ball_Text3.mp4", 
                  "./5Ball/5Ball_Text4.mp4", "./5Ball/5Ball_Text5.mp4"]

seven_ball_none = ["./7Ball/7Ball_No_Screen1.mp4", "./7Ball/7Ball_No_Screen2.mp4", "./7Ball/7Ball_No_Screen3.mp4", 
                   "./7Ball/7Ball_No_Screen4.mp4", "./7Ball/7Ball_No_Screen5.mp4", "./7Ball/7Ball_No_Screen6.mp4", 
                   "./7Ball/7Ball_No_Screen7.mp4"]
seven_ball_arrows = ["./7Ball/7Ball_Arrow1.mp4", "./7Ball/7Ball_Arrow2.mp4", "./7Ball/7Ball_Arrow3.mp4", 
                     "./7Ball/7Ball_Arrow4.mp4", "./7Ball/7Ball_Arrow5.mp4", "./7Ball/7Ball_Arrow6.mp4", 
                     "./7Ball/7Ball_Arrow7.mp4"]
seven_ball_eyes = ["./7Ball/7Ball_Eyes1.mp4", "./7Ball/7Ball_Eyes2.mp4", "./7Ball/7Ball_Eyes3.mp4", 
                   "./7Ball/7Ball_Eyes4.mp4", "./7Ball/7Ball_Eyes5.mp4", "./7Ball/7Ball_Eyes6.mp4", 
                   "./7Ball/7Ball_Eyes7.mp4"]
seven_ball_text = ["./7Ball/7Ball_Text1.mp4", "./7Ball/7Ball_Text2.mp4", "./7Ball/7Ball_Text3.mp4", 
                   "./7Ball/7Ball_Text4.mp4", "./7Ball/7Ball_Text5.mp4", "./7Ball/7Ball_Text6.mp4", 
                   "./7Ball/7Ball_Text7.mp4"]

In [4]:
def get_video_list(num_balls, type_visual):
    mapping = {
        3: {"none": three_ball_none, "arrows": three_ball_arrows, "eyes": three_ball_eyes, "text": three_ball_text},
        5: {"none": five_ball_none, "arrows": five_ball_arrows, "eyes": five_ball_eyes, "text": five_ball_text},
        7: {"none": seven_ball_none, "arrows": seven_ball_arrows, "eyes": seven_ball_eyes, "text": seven_ball_text},
    }
    return mapping[num_balls][type_visual]

In [5]:
def run_experiment(num_balls=3, type_visual="none", num_trials=10, playback_speed=1.0):
    pygame.init()
    screen = pygame.display.set_mode((400, 150))
    pygame.display.set_caption("Press SPACE to start each trial, then 1–N during video")
    font = pygame.font.Font(None, 28)
    
    results = []
    videos = get_video_list(num_balls, type_visual)
    scale = 0.8  # 80% video frame size

    for trial in range(1, num_trials + 1):
        clear_output(wait=True)
        print(f"Trial {trial}/{num_trials} — Press SPACE to start")

        # Wait for SPACE to start
        waiting = True
        while waiting:
            screen.fill((30, 30, 30))
            text = font.render(f"Trial {trial}/{num_trials}: Press SPACE to start", True, (255, 255, 255))
            screen.blit(text, (20, 60))
            pygame.display.flip()
            for event in pygame.event.get():
                if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                    waiting = False
                elif event.type == pygame.QUIT:
                    pygame.quit()
                    return pd.DataFrame(results)

        # Pick a random video and setup
        video_path = random.choice(videos)
        cap = cv2.VideoCapture(video_path)
        start_time = time.time()
        reaction_time = None
        key_pressed = None
        correct = False

        print(f"Playing: {video_path}")

        base_delay = 33  # ~30 fps base
        delay = max(1, int(base_delay / playback_speed))

        # Video playback + key listening loop
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Resize frame to 80%
            height, width = frame.shape[:2]
            new_dim = (int(width * scale), int(height * scale))
            frame = cv2.resize(frame, new_dim)
        
            cv2.imshow("Video", frame)
            key = cv2.waitKey(delay) & 0xFF  # Detect key press directly
        
            # Inside the video loop, when detecting a key press:
            # Extract the correct number from filename
            filename = os.path.basename(video_path)  # e.g., "5Ball_Arrows3.mp4"
            correct_number = int(''.join(filter(str.isdigit, filename.split('.')[-2][-1])))  
            # Explanation: take the last digit before ".mp4"
            
            if key in [ord(str(i)) for i in range(1, num_balls + 1)]:
                key_pressed = int(chr(key))
                reaction_time = time.time() - start_time
                correct = (key_pressed == correct_number)  # compare to filename
            
                results.append({
                    "trial": trial,
                    "video": video_path,
                    "key": key_pressed,
                    "reaction_time": round(reaction_time, 3),
                    "correct": correct,
                    "speed": playback_speed
                })
            
                print(f"Trial {trial}: Key {key_pressed}, RT={reaction_time:.3f}s, "
                      f"{'Correct (advancing...)' if correct else '❌ Incorrect (waiting...)'}")
            
                if correct:
                    break  # advance to next trial


            if key_pressed is not None:
                break

        cap.release()
        cv2.destroyAllWindows()

    pygame.quit()
    df = pd.DataFrame(results)
    return df

In [17]:
# Execute cell to start trial. Controls:
# - num_balls : total number of balls in trials. Options are 3, 5, and 7
# - type_visual : choose from none, arrows, eyes, text - Changes what is displayed.
# - num_trials : number of tests to run.

# How this works: Select the testing mode and number of balls. Run test. Record reaction time and % correct. Make observations based on results.

# - Space to start

# Questions to ask after tests:
# 1. How friendly do you view the robot on a scale of 1 - 7?
# 2. On a scale of 1 - 7, how much did you rely on the display to determine your answer?
# 3. How helpful did you find the display in determining the robot's actions, from a scale from 1 - 7?
# --- Other questions can be asked in conjunction


results_df = run_experiment(num_balls=7, type_visual="text", num_trials=10, playback_speed=0.8)
results_df.to_csv('new_data.csv', index=False)
results_df

Trial 40/40 — Press SPACE to start
Playing: ./7Ball/7Ball_Text7.mp4
Trial 40: Key 7, RT=3.712s, Correct (advancing...)


,trial,video,key,reaction_time,correct,speed
0,1,./7Ball/7Ball_Text2.mp4,2,2.879,True,0.8
1,2,./7Ball/7Ball_Text5.mp4,5,3.720,True,0.8
2,3,./7Ball/7Ball_Text1.mp4,1,1.464,True,0.8
3,4,./7Ball/7Ball_Text2.mp4,2,3.168,True,0.8
4,5,./7Ball/7Ball_Text4.mp4,4,3.496,True,0.8
5,6,./7Ball/7Ball_Text1.mp4,1,1.912,True,0.8
6,7,./7Ball/7Ball_Text3.mp4,3,3.872,True,0.8
7,8,./7Ball/7Ball_Text2.mp4,2,3.169,True,0.8
8,9,./7Ball/7Ball_Text1.mp4,1,2.609,True,0.8
9,10,./7Ball/7Ball_Text5.mp4,5,4.801,True,0.8


In [7]:
results_df.to_csv('new_data.csv', index=False)